# Úkol č. 2 - předzpracování dat a binární klasifikace (do 9. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


import sklearn.metrics as metrics
from sklearn.model_selection import ParameterGrid

random_seed = 727

In [2]:
def encode_categories(df, mappers, dummies=False):
    label_encoder = LabelEncoder()
    for col in df.select_dtypes('object').columns:
        if col not in mappers:
            df[col] = df[col].fillna('NaN')
            df[col] = label_encoder.fit_transform(df[col])
            if dummies:
                df = pd.concat([
                    df.drop(columns=[col]), pd.get_dummies(df[col], prefix=('d_' + col))
                ], axis=1)
        elif col in mappers:
            df[col] = df[col].replace(mappers[col])
    return df

def preprocess(data):
    #data['name'] = data['name'].apply(lambda x: len(x))
    #data = data.rename(columns = {'name':'nameLen'})
    data = data.drop(columns = ['name', 'home.dest', 'ticket'])
    #-----------------------------------------
    data['sex'] = data['sex'].replace({'male': 0, 'female':1})
    #-----------------------------------------
    data['age'] = data['age'].fillna(-1).astype('int64')
    #-----------------------------------------
    data['cabin'] = data['cabin'].fillna('-1')
    #-----------------------------------------
    data['fare'] = data['fare'].fillna(0).apply(lambda x: x*10000).astype('int64')
    #-----------------------------------------
    data['embarked'] = data['embarked'].fillna(0).replace({'S' : 1, 'C':2, 'Q' : 3})
    data = encode_categories(data, {}, dummies=False)
    return data

In [3]:
data = pd.read_csv("data.csv")
evaluation = pd.read_csv("evaluation.csv")

data = preprocess(data)
evaluation = preprocess(evaluation)
#-----------------------------------------
display(data.head())
display(data.isnull().any().any())
data.info()

,ID,survived,pclass,sex,age,sibsp,parch,fare,cabin,embarked
0,0,1,3,0,19,0,0,80500,0,1
1,1,1,2,1,40,0,0,130000,0,1
2,2,0,3,1,18,0,0,67500,0,3
3,3,0,3,0,-1,1,9,695500,0,1
4,4,0,3,1,30,0,0,86625,0,1


False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        1000 non-null   int64
 1   survived  1000 non-null   int64
 2   pclass    1000 non-null   int64
 3   sex       1000 non-null   int64
 4   age       1000 non-null   int64
 5   sibsp     1000 non-null   int64
 6   parch     1000 non-null   int64
 7   fare      1000 non-null   int64
 8   cabin     1000 non-null   int64
 9   embarked  1000 non-null   int64
dtypes: int64(10)
memory usage: 78.2 KB


In [4]:
X_train, X_rest, y_train, y_rest = train_test_split(
    data.drop(columns=['survived']), data['survived'], test_size=0.4, random_state=random_seed
)

X_test, X_val, y_test, y_val = train_test_split(
    X_rest, y_rest, test_size=0.6, random_state=random_seed
)


In [5]:
param_grid = {
    'max_depth': range(1,101), 
    'criterion': ['entropy', 'gini']
}
param_comb = ParameterGrid(param_grid)

val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion']).fit(X_train, y_train)
    train_acc.append(metrics.accuracy_score(y_train, dt.predict(X_train)))
    val_acc.append(metrics.accuracy_score(y_val, dt.predict(X_val)))

best_params = param_comb[np.argmax(val_acc)]
print(best_params)
bestVal = '{0:.6f}'.format(val_acc[np.argmax(val_acc)])
bestTrain = '{0:.6f}'.format(train_acc[np.argmax(val_acc)])
print('best validation accuracy =',bestVal , 'with training accuracy =', bestTrain)

{'max_depth': 29, 'criterion': 'gini'}
best validation accuracy = 0.783333 with training accuracy = 1.000000


In [6]:
dt = DecisionTreeClassifier(**best_params).fit(X_train, y_train)
print('testing accuracy with best params =', '{0:.6f}'.format(metrics.accuracy_score(y_test, dt.predict(X_test))))

testing accuracy with best params = 0.725000


In [7]:
res = dt.predict(evaluation)
resDF = pd.DataFrame(list(zip(evaluation['ID'], res)), 
               columns =['ID', 'survived'])
resDF.to_csv(r'result.csv', index=False)

In [10]:
classifiers = [(DecisionTreeClassifier, {'max_depth': range(1,101), 'criterion': ['entropy', 'gini']}),
               (RandomForestClassifier, {'n_estimators': range(1, 100, 5), 'max_depth': range(1, 5)}),
               (AdaBoostClassifier,  {'n_estimators': range(1,100,5), 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1]}),
               (KNeighborsClassifier, {'n_neighbor': range(2, 100)})]

bestMethod = ('', '', -1)

for method, p in classifiers:
    param_comb = ParameterGrid(p)
    val_acc = []
    train_acc = []
    for param in param_comb:
        dt = method(**param).fit(X_train, y_train)
        
        train_acc.append(metrics.accuracy_score(y_train, dt.predict(X_train)))
        val_acc.append(metrics.accuracy_score(y_val, dt.predict(X_val)))
        
    print('used', method.__name__)
    best_params = param_comb[np.argmax(val_acc)]
    print(best_params)
    bestVal = val_acc[np.argmax(val_acc)]
    print('best validation accuracy =', '{0:.6f}'.format(bestVal))
    print('----------------------------')
    
    if bestVal > bestMethod[2]:
        bestMethod = (method, best_params, bestVal)
        
print('best was', bestMethod[0].__name__, 'with', bestMethod[1])
dt = bestMethod[0](** bestMethod[1]).fit(X_train, y_train)
print('accuracy on test data', metrics.accuracy_score(y_test, dt.predict(X_test)) )


used DecisionTreeClassifier
{'max_depth': 13, 'criterion': 'gini'}
best validation accuracy = 0.791667
----------------------------
used RandomForestClassifier
{'n_estimators': 1, 'max_depth': 3}
best validation accuracy = 0.775000
----------------------------
used AdaBoostClassifier
{'n_estimators': 46, 'learning_rate': 1}
best validation accuracy = 0.754167
----------------------------
best was DecisionTreeClassifier with {'max_depth': 13, 'criterion': 'gini'}
accuracy on test data 0.725
